In [2]:
import cv2
import pytesseract
import numpy as np
import pandas as pd
import time

In [3]:
np.random.seed(42)
nums = np.random.choice(np.arange(300000), size=5000)
now = time.time()
images = [cv2.imread('/kaggle/input/handwriting-recognition/train_v2/train/TRAIN_' + ''.join(['0']*(5-len(str(i)))) + str(i) +'.jpg', 0) for i in nums]
time.time()-now

4.069092512130737

In [4]:
now = time.time()
texts = [pytesseract.image_to_string(image) for image in images]
time.time()-now

1015.1650409698486

In [5]:
np.array(texts).dump(open('ocr_names.npy', 'wb'))

In [10]:
ref_names = np.array(['TRAIN_' + ''.join(['0']*(5-len(str(i)))) + str(i) for i in nums])
ref_names.dump(open('ref_names.npy', 'wb'))

In [22]:
df_true = pd.read_csv('/kaggle/input/handwriting-recognition/written_name_train_v2.csv')

In [32]:
# ref_names = [ref_name+'.jpg' for ref_name in ref_names]
df_ocr = pd.DataFrame({'FILENAME': ref_names, 'ocr_name': texts})

In [33]:
df = df_true.merge(df_ocr, on='FILENAME', how='inner')

In [46]:
import re
df['ocr_name'] = df['ocr_name'].apply(lambda x: re.sub('([^a-z0-9]|prenom|nom)', '', x.lower()))
df['true_name'] = df['IDENTITY'].apply(lambda x: str(x).lower())

In [48]:
df[['ocr_name', 'true_name']].to_csv('ocr_names.csv')